<a href="https://colab.research.google.com/github/yuri-kasahara/Workshop-Tekst-som-data/blob/main/Example_scraper_NIBR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**This is a script with an example of a scraper to obtain a list of hearings held by the Norwegian government with their respective comments.**

The two cells below will load the packages you need to run the scraper.

In [1]:
%%shell
# Ubuntu no longer distributes chromium-browser outside of snap
#
# Proposed solution: https://askubuntu.com/questions/1204571/how-to-install-chromium-without-snap

# Add debian buster
cat > /etc/apt/sources.list.d/debian.list <<'EOF'
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster.gpg] http://deb.debian.org/debian buster main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster-updates.gpg] http://deb.debian.org/debian buster-updates main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-security-buster.gpg] http://deb.debian.org/debian-security buster/updates main
EOF

# Add keys
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A

apt-key export 77E11517 | gpg --dearmour -o /usr/share/keyrings/debian-buster.gpg
apt-key export 22F3D138 | gpg --dearmour -o /usr/share/keyrings/debian-buster-updates.gpg
apt-key export E562B32A | gpg --dearmour -o /usr/share/keyrings/debian-security-buster.gpg

# Prefer debian repo for chromium* packages only
# Note the double-blank lines between entries
cat > /etc/apt/preferences.d/chromium.pref << 'EOF'
Package: *
Pin: release a=eoan
Pin-Priority: 500


Package: *
Pin: origin "deb.debian.org"
Pin-Priority: 300


Package: chromium*
Pin: origin "deb.debian.org"
Pin-Priority: 700
EOF

Executing: /tmp/apt-key-gpghome.Wkqns5VGOJ/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
gpg: key DCC9EFBF77E11517: public key "Debian Stable Release Key (10/buster) <debian-release@lists.debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Executing: /tmp/apt-key-gpghome.UGsK2V1IEF/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
gpg: key DC30D7C23CBBABEE: public key "Debian Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Executing: /tmp/apt-key-gpghome.oSd39bZeKr/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A
gpg: key 4DFAB270CAA96DFA: public key "Debian Security Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1


In [ ]:
!apt-get update
!apt-get install chromium chromium-driver
!pip3 install selenium

import regex as re
import requests
import time
import pandas as pd
import os
import logging
from bs4 import BeautifulSoup as BS
import urllib.request

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service

options = Options()
options.add_argument("--headless")
options.add_argument("--no-sandbox")

service_driver = Service(executable_path=r'/usr/bin/chromedriver')

driver = webdriver.Chrome(service=service_driver, options=options)


Hit:1 http://deb.debian.org/debian buster InRelease
Hit:2 http://deb.debian.org/debian buster-updates InRelease
Hit:3 http://deb.debian.org/debian-security buster/updates InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:5 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:7 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:8 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:9 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:10 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:12 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:13 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
Reading package lists... Done
Building dependency t

Now, let's get some data!

In [ ]:
 ## This is a link for one of the pages containing the list of hearings.

 url = "https://www.regjeringen.no/no/dokument/hoyringar/oversyn-over-hoyringssaker/id546535/?page=34"

In [ ]:
## We have first to create empty lists to hold our scraped content

Hoeringsnavn = []
Url = []
Departement = []
Utsendt = []
Frist = []
Dager = []
Status = []

## Then we have to ask load our 'driver' and tell it what we want to collect
driver.get(url)
html = driver.page_source

## You can use the developer tools of your web browser to help you identify the areas you want to scrape. You will probably also have to introduce some loops to properly separate each entry you want.
soup = BS(html, "html.parser")
table_contents = soup.find('tbody')
rows = table_contents.find_all('tr')
tds = table_contents.find_all('td')
for r in rows:
# These lines give you the name of the hearing
        n = r.find('h3', {'class' : 'hoering-title'})
        Hoeringsnavn.append(n.text.strip())
# These lines give you the link for the webpage of the hearing
        u = r.find('h3', {'class' : 'hoering-title'})
        lenken = u.find('a')['href']
        lenken = "https://www.regjeringen.no"+str(lenken)
        Url.append(lenken)
# These will give you the name of the ministry responsible for the hearing
        d = r.find('h3', {'class' : 'hoering-title'}).find_next('td').text
        Departement.append(d)
# These will give you the date it was published
        ut = r.find('td', {'class' : 'date'})
        utdato = ut.text.strip()
        Utsendt.append(utdato)
# These will give the deadline
        f = r.find('td', {'class' : 'date'}).find_next_sibling()
        fr = f.text.strip()
        Frist.append(fr)
#These will collect the duration of the hearing in days
        da = r.find('td', {'class' : 'type'})
        Dager.append(da.text.strip())
# These will give you the status
        fer = r.find('td', {'class' : 'type'}).find_next_sibling()
        Status.append(fer.text.strip())
        time.sleep(1)
# Then you have to put all the text collected in a dataframe format by turning the lists into variables.
Hoerings_data = {}
Hoerings_data['Hoeringsnavn'] = Hoeringsnavn
Hoerings_data['Url'] = Url
Hoerings_data['Departement'] = Departement
Hoerings_data['Utsendt'] = Utsendt
Hoerings_data['Frist'] = Frist
Hoerings_data['Dager'] = Dager
Hoerings_data['Status'] = Status

# Here you will make the dataframe
df = pd.DataFrame.from_dict(Hoerings_data, orient='columns')

print(df)

# Now save it on your drive or on your computer
df.to_csv('drive/MyDrive/WorkshopTSD/Hoerings_list.csv', index=False)


                                         Hoeringsnavn  \
0   Høring - forlengelse av midlertidige endringer...   
1   Høring - tverrfaglig helsekartlegging av barn ...   
2   Høring om endringer i alkoholforskriften – Vin...   
3   Høring – forslag til endringer i forskrift om ...   
4   Høring - Forslag til forlengelse av midlertidi...   
5   Høring - samordning av dagpenger mot private t...   
6   Høyring – forslag til endringar i båt- og ferj...   
7   Høring - om endringer i kongekrabbeforskriften...   
8   Høring - utredning om pensjon fra første krone...   
9   Høring - forslag om å gjøre allmenngjøringslov...   
10  Høring - oppheving av plikten til å tilby grun...   
11  Høring - forslag til endringer i akuttmedisinf...   
12  Høring - NOU 2020: 13 Private aktører i velfer...   
13  Høring - Forslag til nytt arbeidsmarkedstiltak...   
14  Høring - Om utbetaling av uføretrygd og pensjo...   
15                        Høring - Ny boligsosial lov   
16  Høring – grenseoverskridend

In [ ]:
df_2 = pd.read_csv("drive/MyDrive/WorkshopTSD/Hoerings_list.csv", sep=',', dtype='string')


pd.options.display.max_colwidth = 150

print(df_2)

                                                                                                                                             Hoeringsnavn  \
0                                             Høring - forlengelse av midlertidige endringer i smittevernloven (oppholdssted under innreisekarantene mv.)   
1                                                                                             Høring - tverrfaglig helsekartlegging av barn i barnevernet   
2                                                                        Høring om endringer i alkoholforskriften – Vinmonopolets innførsel av vareprøver   
3   Høring – forslag til endringer i forskrift om pasient- og brukerrettigheter i fastlegeordningen – rett til å stå på liste hos fastlege for utenlan...   
4                             Høring - Forslag til forlengelse av midlertidige endringer i plan- og bygningsloven for å avhjelpe konsekvenser av covid-19   
5                                                         

In [ ]:
URLs_hoering = df_2['Url']
print(URLs_hoering)

0     https://www.regjeringen.no/no/dokumenter/horing-forlengelse-av-midlertidige-endringer-i-smittevernloven-oppholdssted-under-innreisekarantene-mv/id...
1                                             https://www.regjeringen.no/no/dokumenter/horing-tverrfaglig-helsekartlegging-av-barn-i-barnevernet/id2819979/
2                        https://www.regjeringen.no/no/dokumenter/horing-om-endringer-i-alkoholforskriften-vinmonopolets-innforsel-av-vareprover/id2831889/
3     https://www.regjeringen.no/no/dokumenter/horing-forslag-til-endringer-i-forskrift-om-pasient-og-brukerrettigheter-i-fastlegeordningen-rett-til-a-s...
4     https://www.regjeringen.no/no/dokumenter/horing-forslag-til-forlengelse-av-midlertidige-endringer-i-plan-og-bygningsloven-for-a-avhjelpe-konsekven...
5                                  https://www.regjeringen.no/no/dokumenter/horing-samordning-av-dagpenger-mot-private-tjenestepensjonsordninger/id2831122/
6            https://www.regjeringen.no/no/dokumenter/horing-for

Now, let's get data on the individual contents of each hearing.

The scraper below will use the links from the previous dataset.

In [ ]:
ID = []
Title = []
Short_description = []
Result = []
#Status = []
Deadline = []
Letter = []
Participants = []
Feedback = []
Link_legal_result = []

ID_notater =[]
Links = []

Hoering_organization = []
ID_org = []

ID_svar = []
Comment_org = []
Link_comment = []


for link in URLs_hoering:
    regex = re.compile('id(\d+)')
    i = regex.findall(link)
    ID.append(i)

    print("Getting information from: ", link)
    driver.get(link)
    time.sleep(2)
    html = driver.page_source
    soup = BS(html, "html.parser")

    t = soup.find('header', {'class':'article-header'})
    Title.append(t.text.strip())

    try:
        sd = soup.find('p', {'class':'subheader'})
        Short_description.append(sd.text.strip())
    except:
        Short_description.append(None)

    try:
        res = soup.find('p', {'class':'subheader'})
        Result.append(res.text.strip())

    #except:
        res = soup.find('div', {'class':'article-ingress'})
        Result.append(res.text.strip())

    #try:
        res = soup.find('div', {'id':'horingsresultat'})
        Result.append(res.text.strip())

    except:
        Result.append(None)

    try:
        ls = soup.find('div', {'id':'horingsresultat'})
        lenken = ls.find('a')['href']
        lenken = "https://www.regjeringen.no"+str(lenken)
        Link_legal_result.append(lenken)

    except:
        Link_legal_result.append(None)

    try:
        st = soup.find('div', {'class':'horing-meta content-row'}).text.strip()
        regex_2 = re.compile("\d{2}.\d{2}.\d{4}")
        dt = regex_2.findall(st)
        Deadline.append(dt)

    except:
        Deadline.append(None)

    try:
        br = soup.find('div', {'id':'horingsbrev'}).text.strip()
        Letter.append(br)

    except:
        Letter.append(None)

    try:
        nt = soup.find('div', {'id':'horingsnotater'}).find_all('a', href=True)
        for line in nt:
            link = line['href']
            lenken = "https://www.regjeringen.no"+str(link)
            Links.append(lenken)
            ID_notater.append(i)
    except:
            Links.append(None)
            ID_notater.append(i)

    try:
        pt = soup.find('div', {'id':'horingsinstanser'})
        for br in pt.find_all('br'):
            br.replace_with("\n")
        names = pt.text.strip()
        for org in names.split('\n'):
            Hoering_organization.append(org)
            ID_org.append(i)
    except:
            Hoering_organization.append(None)
            ID_org.append(i)

    try:
        sv = soup.find('div', {'id':'horingssvar'}).find_all('a', href=True)
        for line in sv:
            link = line['href']
            lenken = "https://www.regjeringen.no"+str(link)
            Link_comment.append(lenken)
            org = line.text.strip()
            Comment_org.append(org)
            ID_svar.append(i)
    except:
        Link_comment.append(None)
        Comment_org.append(None)
        ID_svar.append(i)

Hoerings_individual_content_data = {}
Hoerings_individual_content_data['ID'] = ID
Hoerings_individual_content_data['Title'] = Title
Hoerings_individual_content_data['Short_description'] = Short_description
Hoerings_individual_content_data['Result'] = Result
Hoerings_individual_content_data['Deadline'] = Deadline
Hoerings_individual_content_data['Letter'] = Letter
Hoerings_individual_content_data['Legal_result'] = Link_legal_result

Hoerings_notater = {}
Hoerings_notater['ID_notater'] = ID_notater
Hoerings_notater['Links'] = Links

Hoerings_instanser = {}
Hoerings_instanser['ID_org'] = ID_org
Hoerings_instanser['Hoering_organization'] = Hoering_organization

Hoerings_comments = {}
Hoerings_comments['ID_svar'] = ID_svar
Hoerings_comments['Comment_org'] = Comment_org
Hoerings_comments['Link_comment'] = Link_comment


df = pd.DataFrame.from_dict(Hoerings_individual_content_data, orient='columns')
df2 = pd.DataFrame.from_dict(Hoerings_notater, orient='columns')
df3 = pd.DataFrame.from_dict(Hoerings_instanser, orient='columns')
df4 = pd.DataFrame.from_dict(Hoerings_comments, orient='columns')


df.to_csv('drive/MyDrive/WorkshopTSD/Hoerings_individual_content_data.csv', index=False)
df2.to_csv('drive/MyDrive/WorkshopTSD/Hoerings_notater.csv', index=False)
df3.to_csv('drive/MyDrive/WorkshopTSD/Hoerings_instanser.csv', index=False)
df4.to_csv('drive/MyDrive/WorkshopTSD/Hoerings_comments.csv', index=False)

print(df)
print(df2)
print(df3)
print(df4)


Getting information from:  https://www.regjeringen.no/no/dokumenter/horing-forlengelse-av-midlertidige-endringer-i-smittevernloven-oppholdssted-under-innreisekarantene-mv/id2838755/
Getting information from:  https://www.regjeringen.no/no/dokumenter/horing-tverrfaglig-helsekartlegging-av-barn-i-barnevernet/id2819979/
Getting information from:  https://www.regjeringen.no/no/dokumenter/horing-om-endringer-i-alkoholforskriften-vinmonopolets-innforsel-av-vareprover/id2831889/
Getting information from:  https://www.regjeringen.no/no/dokumenter/horing-forslag-til-endringer-i-forskrift-om-pasient-og-brukerrettigheter-i-fastlegeordningen-rett-til-a-sta-pa-liste-hos-fastlege-for-utenlandske-diplomater-mv/id2829961/
Getting information from:  https://www.regjeringen.no/no/dokumenter/horing-forslag-til-forlengelse-av-midlertidige-endringer-i-plan-og-bygningsloven-for-a-avhjelpe-konsekvenser-av-covid-19/id2838405/
Getting information from:  https://www.regjeringen.no/no/dokumenter/horing-samordnin

Now let's collect comments from one hearing.

The scraper below will give you the list of contributors to the hearing and the respective links to the individual contributions.

In [ ]:
url = 'https://www.regjeringen.no/no/dokumenter/horing-tverrfaglig-helsekartlegging-av-barn-i-barnevernet/id2819979/?showSvar=true&term=&page=1&isFilterOpen=true'

ID = []
Name_organization = []
Link_for_comment = []

regex = re.compile('id(\d+)')
i = regex.findall(url)


print("Getting information from: ", url)
driver.get(url)
time.sleep(3)
html = driver.page_source
soup = BS(html, "html.parser")

try:
        sv = soup.find('ul', {"class":"link-list"}).find_all('a', href=True)
        for line in sv:
            link = line['href']
            lenken = "https://www.regjeringen.no/no/dokumenter/horing-tverrfaglig-helsekartlegging-av-barn-i-barnevernet/id2819979/" + str(link)
            l = lenken.strip("('')")
            Link_for_comment.append(l)
            org = line.text.strip()
            Name_organization.append(org)
            ID.append(i)

except:
        Link_for_comment.append(None)
        Name_organization.append(None)
        ID.append(i)



Link_comments = {}
Link_comments['ID'] = ID
Link_comments['Name_organization'] = Name_organization
Link_comments['Link_for_comment'] = Link_for_comment

df_5 = pd.DataFrame.from_dict(Link_comments, orient='columns')

df_5.to_csv('drive/MyDrive/WorkshopTSD/Link_comments.csv', index=False)

driver.quit()

print(df_5)

Getting information from:  https://www.regjeringen.no/no/dokumenter/horing-tverrfaglig-helsekartlegging-av-barn-i-barnevernet/id2819979/?showSvar=true&term=&page=1&isFilterOpen=true
           ID                                  Name_organization  \
0   [2819979]                 Actis - Rusfeltets samarbeidsorgan   
1   [2819979]                                         ADHD Norge   
2   [2819979]                                  Advokatforeningen   
3   [2819979]                                Akademikerforbundet   
4   [2819979]  Akershus universitessykehus, seksjon Stillasby...   
..        ...                                                ...   
95  [2819979]               Statsforvalteren i Troms og Finnmark   
96  [2819979]                       Statsforvalteren i Trøndelag   
97  [2819979]            Stavanger barneverntjeneste (PDF, 29KB)   
98  [2819979]                   Stiftelsen Kirkens Bymisjon Oslo   
99  [2819979]       Stiftelsen Kirkens Bymisjon Oslo (PDF, 22KB)   

 

Now, let's collect the actual comments. This code will give you only the comments published directly in the webpage - not those sent in a pdf format. We will deal with them later.

In [ ]:
#scrape individual comments on the proposals put for hearing

df_5 = pd.read_csv("drive/MyDrive/WorkshopTSD/Link_comments.csv", sep=',', dtype='string')

URLs_comments = df_5['Link_for_comment']

ID_hoering = df_5['ID']

Comment_org = df_5['Name_organization']

ID = []

Text_comment = []

Organisation = []

Comment_links_revised = {}

full_text = ""

for link, i, org in zip(URLs_comments, ID_hoering, Comment_org):
        print("Getting information from: ", link)



        try:
                   driver.get(link)
                   html = driver.page_source
                   soup = BS(html, "html.parser")
                   time.sleep(3)
                   t = soup.find('div', {'class' : 'article-body'}).find_all('p')
                   print(t)
                   for paragraph in t:
                            text = paragraph.text.strip()
                            full_text += text + " "
                   Text_comment.append(full_text)
                   full_text = ""
                   ID.append(i)
                   Organisation.append(org)



        except:
                   Text_comment.append(None)
                   ID.append(i)
                   Organisation.append(org)






Comment_links_revised['ID'] = ID
Comment_links_revised['Text_comment'] = Text_comment
Comment_links_revised['Organisation'] = Organisation

df = pd.DataFrame.from_dict(Comment_links_revised, orient='columns')

df.to_csv('drive/MyDrive/WorkshopTSD/Comment_links.csv', index=False)



Getting information from:  https://www.regjeringen.no/no/dokumenter/horing-tverrfaglig-helsekartlegging-av-barn-i-barnevernet/id2819979/?uid=3223d55d-af0c-4c99-b898-04c9dccffbf6
[]
Getting information from:  https://www.regjeringen.no/no/dokumenter/horing-tverrfaglig-helsekartlegging-av-barn-i-barnevernet/id2819979/?uid=ea6060f9-31a0-4912-a2a5-4ed3f30a5dbf
[<p><strong><span>Høring tverrfaglig helsekartlegging av barn i barnevernet</span></strong></p>, <p><span> </span></p>, <p><span>ADHD Norge arbeider for å spre kunnskap om ADHD for medlemmer, hjelpeapparatet, undervisningssektor og i befolkningen generelt. ADHD kjennetegnes av hyperaktivitet, konsentrasjonsvansker og impulsivitet og<span> forekommer hos 3-5 prosent av barn i skolealder</span></span><span>.</span></p>, <p>ADHD Norge støtter forslaget om tverrfaglig helsekartlegging av barn i barnevernet etter samtykke, men har noen kommentarer.</p>, <p>Det er svært positivt at det skal utarbeides nasjonale anbefalinger om innholdet i 